# Evaluate and fit a ML model on the EC flux tower data 



## Load modules

In [ ]:
import os
import shap
import xarray as xr
import numpy as np
import pandas as pd
import seaborn as sb
from joblib import dump
import multiprocessing
from pprint import pprint
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV, ShuffleSplit, KFold
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

import warnings
warnings.filterwarnings("ignore")

## Analysis Parameters

In [ ]:
model_name = 'AUS'
model_var = 'NEE'

In [ ]:
ncpus=multiprocessing.cpu_count()
print('ncpus = '+str(ncpus))

## Prepare Data

In [ ]:
base = '/g/data/os22/chad_tmp/NEE_modelling/results/training_data/'
sites = os.listdir('/g/data/os22/chad_tmp/NEE_modelling/results/training_data/')

td = []
for site in sites:
    if '.csv' in site:
        xx = pd.read_csv(base+site, index_col='time', parse_dates=True)
        td.append(xx)

In [ ]:
variables = [
             # 'LAI_anom_RS',
             'kNDVI_anom_RS',
             'FPAR_RS',
             #'LST_RS',
             'tree_cover_RS',
             'nontree_cover_RS',
             'nonveg_cover_RS',
             'LST-Tair_RS',
             'TWI_RS',
             'NDWI_RS',
             'rain_anom_RS',
             'rain_cml3_anom_RS',
             'rain_cml6_anom_RS',
             'rain_cml12_anom_RS',
             'srad_anom_RS',
             'vpd_RS',
             'tavg_anom_RS',
             'SOC_RS',
             #'CO2_RS'
            ]

## Modelling

In [ ]:
xx = []
yy = []
for t in td:
    #t = t.drop('PFT_RS', axis=1)  
    t = t.dropna()  # remove NaNS
    df = t.drop(['NEE_SOLO_EC','GPP_SOLO_EC','ER_SOLO_EC'], axis=1) # seperate carbon fluxes
    
    #df = df.filter(regex='RS') # only use remote sensing variables   
    df = df[variables]
    
    # Write out predictior variables to text file
    textfile = open("/g/data/os22/chad_tmp/NEE_modelling/results/variables.txt", "w")
    for element in df.columns:
        textfile.write(element + ",")
    textfile.close()
    
    if model_var == 'ET':
        df_var=t[model_var+'_EC']
    else:
        df_var=t[model_var+'_SOLO_EC'] # seperate out the variable we're modelling

    x = df.reset_index(drop=True)#.to_numpy()
    y = df_var.reset_index(drop=True)#.to_numpy()
    xx.append(x)
    yy.append(y)

x = np.concatenate([x for x in xx])
y = np.concatenate([y for y in yy])

print(x.shape)

## Feature Selection

In [ ]:
from sklearn.feature_selection import SelectFromModel #RFE,mutual_info_regression,SelectKBest,SequentialFeatureSelector, 

In [ ]:
num_of_features = 10

In [ ]:
model = RandomForestRegressor()

# possible feature selection methods
selector = SelectFromModel(
    estimator=model, threshold=-np.inf, max_features=num_of_features
)

# selector = RFE(
#     estimator=model, n_features_to_select=num_of_features
# )

# selector = SelectKBest(
#     mutual_info_regression, k=num_of_features
# )

# selector = SequentialFeatureSelector(
#     model, n_features_to_select=num_of_features, direction="backward", n_jobs=ncpus
# )

In [ ]:
selected_features = selector.fit_transform(
    x, y
)

# Print the name of the features selected
idx_of_selections = selector.get_support(indices=True)
selected_columns = list(np.array(variables)[idx_of_selections])

print("Features selected:")
print(selected_columns)


## Test model robustness with nested K-fold cross validation

In [ ]:
inner_cv_splits = 5

outer_cv_splits = 5

test_size = 0.20

In [ ]:
# Create the parameter grid based on the results of random search 
param_grid = {
    # 'criterion': ["squared_error", "absolute_error"],
    'max_features': [1.0, 'log2', None],
    'n_estimators': [100,200,300,400]
}

In [ ]:
outer_cv = KFold(n_splits=outer_cv_splits, shuffle=True,
                        random_state=0)

# lists to store results of CV testing
acc = []
rmse=[]
r2=[]
i = 1
for train_index, test_index in outer_cv.split(x, y):
    print(f"Working on {i}/5 outer cv split", end='\r')
    model = RandomForestRegressor(random_state=1, n_jobs=ncpus)

    # index training, testing, and coordinate data
    X_tr, X_tt = x[train_index, :], x[test_index, :]
    y_tr, y_tt = y[train_index], y[test_index]
    
    # inner split on data within outer split
    inner_cv = KFold(n_splits=inner_cv_splits,
                     shuffle=True,
                     random_state=0)
    
    clf = GridSearchCV(
        estimator=model,
        param_grid=param_grid,
        scoring='r2',
        n_jobs=ncpus,
        refit=True,
        cv=inner_cv.split(X_tr, y_tr),
    )

    clf.fit(X_tr, y_tr)
    
    # predict using the best model
    best_model = clf.best_estimator_
    pred = best_model.predict(X_tt)

    # evaluate model w/ multiple metrics
    # r2
    r2_ = r2_score(y_tt, pred)
    r2.append(r2_)
    # MAE
    ac = mean_absolute_error(y_tt, pred)
    acc.append(ac)
    # RMSE
    rmse_ = np.sqrt(mean_squared_error(y_tt, pred))
    rmse.append(rmse_)
    i += 1

In [ ]:
print("Mean MAE accuracy: "+ str(round(np.mean(acc), 2)))
print("Std dev of MAE accuracy: "+ str(round(np.std(acc), 2)))
print('\n')
print("Mean RMSE: "+ str(round(np.mean(rmse), 2)))
print("Std dev RMSE: "+ str(round(np.std(rmse), 2)))
print('\n')
print("Mean r2: "+ str(round(np.mean(r2), 2)))
print("Std dev r2: "+ str(round(np.std(r2), 2)))

## Optimize hyperparameters

In [ ]:
param_grid = {
    'max_depth': [None, 10, 20, 30, 40, 50],
    'max_features': ['log2', None, "sqrt"],
    'n_estimators': [200,300,400]
}

In [ ]:
#generate n_splits of train-test_split
rs = ShuffleSplit(n_splits=outer_cv_splits, test_size=test_size, random_state=1)

#instatiate a gridsearchCV
clf = GridSearchCV(RandomForestRegressor(),
                   param_grid,
                   scoring='r2',
                   verbose=1,
                   cv=rs.split(x, y),
                   n_jobs=ncpus)

clf.fit(x, y)

In [ ]:
print("The most accurate combination of tested parameters is: ")
pprint(clf.best_params_)
print('\n')
print("The r2 score using these parameters is: ")
print(round(clf.best_score_, 2))

## Fit on all data using best params

In [ ]:
model = RandomForestRegressor(**clf.best_params_, random_state=1, n_jobs=ncpus)
model.fit(x, y)

## Save the model

In [ ]:
dump(model, '/g/data/os22/chad_tmp/NEE_modelling/results/models/'+model_name+'_'+model_var+'_RF_model.joblib')

## Predict on all the data

In [ ]:
y_pred = model.predict(x)

rmse = float(format(np.sqrt(mean_squared_error(y, y_pred)), '.3f'))
print("RMSE:", rmse);

In [ ]:
sb.scatterplot(x=y,y=y_pred,color="#338844", edgecolor="white", s=50, lw=1, alpha=0.5)
sb.regplot(x=y, y=y_pred, scatter=False, color='m')
sb.regplot(x=y, y=y, color='black', scatter=False, line_kws={'linestyle':'dashed'});
plt.xlabel('True')
plt.ylabel('Prediction');

## Examine feature importance

SHAP (SHapley Additive exPlanations) is a game theoretic approach to explain the output of any machine learning model

https://github.com/slundberg/shap

https://towardsdatascience.com/explaining-scikit-learn-models-with-shap-61daff21b12a

In [ ]:
# explain the model's predictions using SHAP
explainer = shap.Explainer(model)
shap_values = explainer(x)

In [ ]:
#add names of features
shap_values.feature_names = df.columns.values

# visualize the importances
# shap.plots.waterfall(shap_values[0])
# shap.plots.beeswarm(shap_values)
shap.plots.bar(shap_values)